<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EB%B2%88%EC%97%AD%EA%B8%B0(%ED%95%9C%EA%B5%AD_%EC%98%81%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision

In [2]:
#https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 19.4MB 1.1MB/s 
     |████████████████████████████████| 3.8MB 55.0MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab_kor = mecab
spacy_en = spacy.load('en')

In [4]:
mecab_kor.morphs('나는 한국인 입니다')



['나', '는', '한국인', '입니다']

In [5]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_kor(text):
    """
    Tokenizes Korean text from a string into a list of strings (tokens) and reverses it
    """
    return mecab_kor.morphs(text)[::-1]

In [7]:
tokenize_en('I am a boy')

['I', 'am', 'a', 'boy']

In [8]:
SRC = Field(tokenize = tokenize_kor, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [9]:
import pandas as pd
news_df = pd.read_excel('/content/drive/My Drive/B반/data/kor.xlsx', sheet_name='Sheet1') 
train_df, val_df, test_df = news_df.iloc[:50000, 1:], news_df.iloc[50000:63000, 1:], news_df.iloc[63000:, 1:]

train_df.to_csv('/content/drive/My Drive/Seq to Seq/train_df.tsv', sep='\t', index=False)
test_df.to_csv('/content/drive/My Drive/Seq to Seq/test_df.tsv', sep='\t', index=False)
val_df.to_csv('/content/drive/My Drive/Seq to Seq/val_df.tsv', sep='\t', index=False)

In [10]:
!cat '/content/drive/My Drive/Seq to Seq/train_df.tsv'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
우리는 친근한 회사로 인식되기를 원해요.	We want to be recognized as a friendly company.
한국어 가능한 선생님을 원해요.	We want a teacher who is possible for Korean.
귀사에 무궁한 발전을 기원해요.	I wish you tremendous success in everything your company do.
그들은 가능한 빨리 데이터와 부품을 얻길 원해요.	They want to get data and materials as soon as possible.
남편은 졸업한 후에 박사 과정을 하기를 원해요.	My husband wants be in the doctoral course after he graduates.
희망이 가득한 새해가 되길 기원해요.	I hope that it would be a happy new year that is full of hope.
내 자녀 또한 그 누구보다도 행복해지길 간절히 원해요.	I also hope my children become happier than anyone.
저는 발레 학원에 다니길 원해요.	I want to go to the ballet academy.
나는 많은 학생 들이 철들기를 원해요.	I want lots of students to grow up.
나는 우리 학교에 네 과목을 개설하기 원해요.	I want to open your course in our school.
나는 우리 학생 들이 오른쪽 뇌의 기능을 가지는 것을 원해요.	I want my students to have the function of the right brain.
그가 영어 학원을 가기 원해요.	He wants to go to english academy.
나는 내가 하고 싶은 말을 할수있길 원해요.	I want you to say what you want to say.
어떤 것을 하든지 우리는 

In [11]:
from torchtext.data import TabularDataset

# train_data, test_data = TabularDataset.splits(
#         path='.', train='train_data.csv', test='test_data.csv', format='csv',
#         fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

train_data, valid_data, test_data = TabularDataset.splits(path='.',
					train='/content/drive/My Drive/Seq to Seq/train_df.tsv',
					validation='/content/drive/My Drive/Seq to Seq/val_df.tsv',
					test='/content/drive/My Drive/Seq to Seq/test_df.tsv',
					format='tsv', 
					fields=[('src', SRC), ('trg', TRG)], skip_header=True)
# train_data, test_data = TabularDataset.splits(
#         path='.', train='/content/drive/My Drive/Seq to Seq/train_df.csv', test='/content/drive/My Drive/Seq to Seq/test_df.csv', format='csv',
#         fields=[('ko', SRC), ('en', TRG)])#, skip_header=True)

In [12]:
valid_data

In [13]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 50000
Number of validation examples: 13000
Number of testing examples: 12000


In [14]:
print(vars(train_data.examples[0]))
print(vars(valid_data.examples[0]))
print(vars(test_data.examples[0]))

{'src': ['.', '가요', '으로', '다락방', '러', '만나', '를', '배트', '저녁', '매일', '는', '나'], 'trg': ['i', 'go', 'to', 'the', 'attic', 'every', 'evening', 'to', 'meet', 'bat', '.']}
{'src': ['.', '에요', '이', '여행', '세계', '은', '계획', '의', '꿈', '는', '원하', '에', '여름', '이번', '가', '내'], 'trg': ['for', 'this', 'summer', ',', 'i', 'have', 'a', 'dream', 'to', 'travel', 'abroad', '.']}
{'src': ['.', '해요', '고자', '하', '말', '대해', '에', '이유', '그', '오늘', '는', '저'], 'trg': ['and', 'i', "'m", 'here', 'to', 'explain', 'the', 'reason', 'for', 'this', '.']}


In [15]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [16]:
print(f"Unique tokens in source (kr) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (kr) vocabulary: 10064
Unique tokens in target (en) vocabulary: 8461


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
BATCH_SIZE = 128

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
#     (train_data, valid_data, test_data), 
#     batch_size = BATCH_SIZE, 
#     device = device,
#     sort_key = lambda x: len(x.values()),
#     sort_within_batch=False)


In [19]:
train_iterator  = BucketIterator(
    train_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [20]:
valid_iterator  = BucketIterator(
    valid_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [21]:
test_iterator  = BucketIterator(
    test_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [22]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [23]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [24]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [25]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [26]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10064, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8461, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8461, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,439,309 trainable parameters


In [28]:
optimizer = optim.Adam(model.parameters())

In [29]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [30]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
N_EPOCHS = 150
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 49s
	Train Loss: 4.270 | Train PPL:  71.488
	 Val. Loss: 5.012 |  Val. PPL: 150.176
Epoch: 02 | Time: 0m 49s
	Train Loss: 4.096 | Train PPL:  60.129
	 Val. Loss: 4.878 |  Val. PPL: 131.335
Epoch: 03 | Time: 0m 50s
	Train Loss: 3.915 | Train PPL:  50.149
	 Val. Loss: 4.780 |  Val. PPL: 119.163
Epoch: 04 | Time: 0m 49s
	Train Loss: 3.773 | Train PPL:  43.530
	 Val. Loss: 4.745 |  Val. PPL: 114.977
Epoch: 05 | Time: 0m 50s
	Train Loss: 3.635 | Train PPL:  37.894
	 Val. Loss: 4.645 |  Val. PPL: 104.027
Epoch: 06 | Time: 0m 50s
	Train Loss: 3.519 | Train PPL:  33.743
	 Val. Loss: 4.651 |  Val. PPL: 104.650
Epoch: 07 | Time: 0m 50s
	Train Loss: 3.364 | Train PPL:  28.915
	 Val. Loss: 4.599 |  Val. PPL:  99.425
Epoch: 08 | Time: 0m 50s
	Train Loss: 3.257 | Train PPL:  25.983
	 Val. Loss: 4.598 |  Val. PPL:  99.264
Epoch: 09 | Time: 0m 50s
	Train Loss: 3.140 | Train PPL:  23.111
	 Val. Loss: 4.591 |  Val. PPL:  98.600
Epoch: 10 | Time: 0m 50s
	Train Loss: 3.041 | Train PPL

In [36]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.564 | Test PPL:  95.946 |


In [37]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.568 | Test PPL:  96.311 |
